# December 23, 2023

https://adventofcode.com/2023/day/23

In [3]:
text = f'''#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#'''

test_text = text.split("\n")

In [1]:
fn = "../data/2023/23.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz_text = [x.strip() for x in text]

In [126]:
class Forest:
    def __init__( self, text ):
        self.text = text
        
        self.width = len(text[0])
        self.height = len(text)
        self.start = (0,1)
        self.goal = (self.height-1, self.width-2)
        self.solutions = {}

        self.paths = self.hubs = None

    def symbol( self, rc ):
        '''return symbol on map at location rc=[row,col]'''
        r,c = rc
        if (c<0 or c==self.width or c<0 or c==self.height):
            return None
        return self.text[r][c]
    
    def slope_okay( self, rc0, rc1 ):
        '''
        check if we can go from rc0 to rc1 if the slope at rc1 is slippery
        
        assumes rc0 is non-diagonally adjacent to rc1
        '''
        sym = self.symbol( rc1 )
        
        if sym not in ["<", ">", "v", "^"]:
            raise BaseException(f'''{rc1[0]}, {rc1[1]} is not a slope''')

        # going up
        if rc1[0] == rc0[0] - 1:
            return sym != "v"
        # going down
        if rc1[0] == rc0[0] + 1:
            return sym != "^"
        # going left
        if rc1[1] == rc0[1] - 1:
            return sym != ">"
        # going right
        if rc1[1] == rc0[1] + 1:
            return sym != "<"
        
    def map_paths( self ):
        '''determine all the long windy paths as well as their neighbors'''

        # keyed by start space, value is the end of a previously traced path
        hubs_to_explore = [ [0,0] ]
        num_explored = 0
        self.paths = {}
        self.hubs = {}

        while (num_explored < len(hubs_to_explore)):
            hub = hubs_to_explore[ num_explored ]
            nbrs = self.find_neighbors( hub )
            self.hubs[ (*hub,) ] = nbrs

            for start in nbrs:
                if (*start,) not in self.paths.keys():
                    steps, end, next_hub, nbrs = self.measure_segment( start, hub )

                    # add path and reverse path to the dict
                    self.paths[ (*start,) ] = {
                        'end': end,
                        'slope': self.symbol(start),
                        'steps': steps, # don't count step unto the hub
                        'hub': next_hub,
                        'nbrs': nbrs
                    }

                    self.paths[ (*end,) ] = {
                        'end': start,
                        'slope': self.symbol(end),
                        'steps': steps, # don't count step unto the hub
                        'hub': hub,
                        'nbrs': [n for n in self.hubs[(*hub,)] if n != start]
                    }

                    if (next_hub is not None and next_hub not in hubs_to_explore):
                        hubs_to_explore.append(next_hub)

            num_explored += 1

    def find_neighbors( self, rc ):
        '''return list of neighbors for the given point'''
        r,c = rc

        radj_list = [-1, 0, 0, +1]
        cadj_list = [0, -1, +1, 0]
        nbrs = []

        for radj, cadj in zip(radj_list, cadj_list):
            look = [r+radj, c+cadj]
            sym = self.symbol( look )
            if (sym is not None) and (sym != "#"):
                nbrs.append( look )

        return nbrs

    def measure_segment( self, cur, prev ):
        '''
        count the number of steps from cur to either the goal or the next decision point
        
        returns step count, end of the segment, and list of neighbors for next move
        '''
        steps = 0
        while True:
            # find open spaces, excluding the previous space we were in
            nbrs = self.find_neighbors(cur)
            nbrs = [ n for n in nbrs if n != prev ]
            if len(nbrs) > 1:
                # don't count the hub in the path length
                return steps-1, prev, cur, nbrs
            
            steps += 1
            prev = cur
            cur = nbrs[0]
            if cur == list(self.goal):
                # if we're at the goal, then it's the path end, hub is None, and no neighbors
                return steps, cur, None, []
    
    def max_dist( self, start=None, slippy=True, hubs_visited=None, verbose=False,
                 calls=[0] ):
        '''return the maximum distance from cur to the goal
        
        if slippy, must follow direction of slopes <>v^
        otherwise, free to go either direction there
        '''

        if self.paths is None:
            # gets self.paths and self.hubs, which I think we don't actually need
            self.map_paths()
        if start is None:
            start = self.start
            calls[0] = 0
            self.solutions = {}

        calls[0] += 1
        if calls[0] % 100000 == 0 or verbose:
            print(calls[0])
            #print(self.solutions)
        
        # # Check memo:
        # hv_enc = self.encode(hubs_visited)
        # if hv_enc in self.solutions.keys():
        #     print("memo")
        #     return self.solutions[hv_enc]


        # history is the list of previously visited hubs to check for cycles
        # Copy the list to avoid overwriting calling function environment           
        if hubs_visited is None:
            hubs_visited = [ [0,0] ]
        else:
            hubs_visited = hubs_visited.copy()

        # Get path info for this space
        here = self.paths[ (*start,) ]

        # oops, this cycles back to a previously visited space (hub)
        # this isn't a solution, because the hub history might have another path that is a solution
        if here['hub'] in hubs_visited:
            return None
        
        # otherwise, let's check out this path!
        hubs_visited.append( here['hub'] )

        # current step count is from start to end of this path segment
        steps = here['steps']

        # we made it!
        if here['end'] == list(self.goal):
            if verbose:
                print("FOUND A SOLUTION!")
                print(steps)
            hubs_visited[-1] = list(self.goal) # replace the null so we can record this
            self.solutions[ self.encode(hubs_visited) ] = steps
                
            return steps

        # Check memo:
        hv_enc = self.encode(hubs_visited)
        if hv_enc in self.solutions.keys():
            #print("memo")
            return self.solutions[hv_enc]




        # otherwise, recurse and see if we can make it
        nbrs = here['nbrs']
        best = 0
        for n in nbrs:
            # if it's not slippy, we should try this path
            # or it's slippy, but in the right direction
            if not slippy or self.slope_okay( here['hub'], n ):
                    # get longest path starting at the neighbor
                    result = self.max_dist( n, slippy=slippy, hubs_visited=hubs_visited, verbose=verbose, calls=calls )
                    if result is not None:
                        remaining_steps = result
                        if remaining_steps > best:
                            best = remaining_steps

            else:
                # slippy and we can't go this direction!
                pass

        # if best is still 0, then we didn't find a solution
        if best == 0:
             self.solutions[self.encode(hubs_visited)] = None
             return None
        
        # if best > 0, then we found a solution!
        # total distance is start path, +1 for hub, +1 to start of remaining path, + remaining path
        best_steps = steps + best + 2
        if verbose:
            print("PATH_SO_FAR")
            print(best_steps)

        self.solutions[ self.encode(hubs_visited) ] = best_steps#, best_path]

        # add this path, plus the recursion steps, plus one for the hub, plus one for the step onto the next path
        return best_steps
    
    def encode(self, hub_list):
        if hub_list is None:
            return None
        j = hub_list.copy()
        #j = j[:-1]
        #j.sort()
        #j.append( hub_list[-1] )
        #j = hub_list
        return " : ".join( [ f'''{h[0]},{h[1]}''' for h in j] )
        #return (*[h[0]*self.width+h[1] for h in hub_list],)
        
    def max_dist_iter( self, start=None, slippy=True, paths=None):
        paths = self.paths.copy()
        last_hub = paths[paths.goal]['hub']
        





### Part 1

In [130]:
test = Forest(test_text)

row1 = "00000000001111111111222"
row2 = "01234567890123456789012"

nums = ["  ", "  "] + [ f'''{i:02d}''' for i in range(23)] + ["  ", "  "]
rows = [row1, row2] + test.text + [row1, row2]

for i in range(len(nums)):
    print(f'''{nums[i]} {rows[i]}''')

   00000000001111111111222
   01234567890123456789012
00 #.#####################
01 #.......#########...###
02 #######.#########.#.###
03 ###.....#.>.>.###.#.###
04 ###v#####.#v#.###.#.###
05 ###.>...#.#.#.....#...#
06 ###v###.#.#.#########.#
07 ###...#.#.#.......#...#
08 #####.#.#.#######.#.###
09 #.....#.#.#.......#...#
10 #.#####.#.#.#########v#
11 #.#...#...#...###...>.#
12 #.#.#v#######v###.###v#
13 #...#.>.#...>.>.#.###.#
14 #####v#.#.###v#.#.###.#
15 #.....#...#...#.#.#...#
16 #.#########.###.#.#.###
17 #...###...#...#...#.###
18 ###.###.#.###v#####v###
19 #...#...#.#.>.>.#.>.###
20 #.###.###.#.###.#.#v###
21 #.....###...###...#...#
22 #####################.#
   00000000001111111111222
   01234567890123456789012


In [131]:
test.map_paths()

In [132]:
test.paths

{(0, 1): {'end': [4, 3],
  'slope': '.',
  'steps': 14,
  'hub': [5, 3],
  'nbrs': [[5, 4], [6, 3]]},
 (4, 3): {'end': [0, 1], 'slope': 'v', 'steps': 14, 'hub': [0, 0], 'nbrs': []},
 (5, 4): {'end': [3, 10],
  'slope': '>',
  'steps': 20,
  'hub': [3, 11],
  'nbrs': [[3, 12], [4, 11]]},
 (3, 10): {'end': [5, 4],
  'slope': '>',
  'steps': 20,
  'hub': [5, 3],
  'nbrs': [[4, 3], [6, 3]]},
 (6, 3): {'end': [12, 5],
  'slope': 'v',
  'steps': 20,
  'hub': [13, 5],
  'nbrs': [[13, 6], [14, 5]]},
 (12, 5): {'end': [6, 3],
  'slope': 'v',
  'steps': 20,
  'hub': [5, 3],
  'nbrs': [[4, 3], [5, 4]]},
 (3, 12): {'end': [10, 21],
  'slope': '>',
  'steps': 28,
  'hub': [11, 21],
  'nbrs': [[11, 20], [12, 21]]},
 (10, 21): {'end': [3, 12],
  'slope': 'v',
  'steps': 28,
  'hub': [3, 11],
  'nbrs': [[3, 10], [4, 11]]},
 (4, 11): {'end': [12, 13],
  'slope': 'v',
  'steps': 22,
  'hub': [13, 13],
  'nbrs': [[13, 12], [13, 14], [14, 13]]},
 (12, 13): {'end': [4, 11],
  'slope': 'v',
  'steps': 22,
 

In [133]:
test.max_dist()

94

In [134]:
puzz = Forest(puzz_text)
puzz.max_dist()

2134

### Part 2

In [129]:
test.max_dist(slippy=False, verbose=False)

memo
memo
memo
memo


154

Wrong answers  
6148 too low  
6298 correct!

In [ ]:
puzz.max_dist(slippy=False)

In [ ]:
# 6148 too low!
puzz.max_dist(slippy=False)